In [14]:
from datasets import load_dataset
dataset = load_dataset("openai_humaneval")

In [15]:
infos = dataset["test"]
function_defs = infos["prompt"]
function_sols = infos["canonical_solution"]

In [16]:
full_code_list = [""]*len(function_defs)
for i, (function_def, function_sol) in enumerate(zip(function_defs, function_sols)):
    full_function = function_def.split('    """')[0] + function_sol
    full_code_list[i] = full_function

In [17]:
from openai_code import explain_code, generate_inputs, generate_unittest_advanced

import yaml

with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)
openai_key = config["openai_key"]

In [38]:
code = full_code_list[6]
print(code)

from typing import List


def parse_nested_parens(paren_string: str) -> List[int]:
    def parse_paren_group(s):
        depth = 0
        max_depth = 0
        for c in s:
            if c == '(':
                depth += 1
                max_depth = max(depth, max_depth)
            else:
                depth -= 1

        return max_depth

    return [parse_paren_group(x) for x in paren_string.split(' ') if x]



In [29]:
from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    result = []
    current_string = []
    current_depth = 0

    for c in paren_string:
        if c == '(':
            current_depth += 1
            current_string.append(c)
        elif c == ')':
            current_depth -= 1
            current_string.append(c)

            if current_depth == 0:
                result.append(''.join(current_string))
                current_string.clear()

    return result

In [30]:
code_explanation = explain_code(code, openai_key)
input_list = generate_inputs(code, code_explanation, openai_key)

In [33]:
print(eval(input_list))

[([], 5), ([1, 2, 3, 4], 0), ([10, 20, 30, 40, 50], -1), ([100, 200, 300], 999), ([-5, -10, -15, -20], -100)]


In [34]:
def execute_function_from_string(func_string, *args, **kwargs):
    # Execute the function definition in the current scope
    exec(func_string)

    # Get the function from locals    
    start_index = func_string.find("def ")
    end_index = func_string.find("(")
    if start_index == -1 or end_index == -1:
        raise ValueError("No function definition found in the provided string.")

    # Extract the function definition substring
    function_definition = func_string[start_index:end_index]
    function_name = function_definition.split("(")[0][4:]
    function_to_call = locals().get(function_name)

    # Check if the function exists
    if function_to_call is not None:
        # Call the function with arguments and return the result
        return function_to_call(*args, **kwargs)
    else:
        raise ValueError("No function named 'function' found in the provided string.")

In [35]:
output_list = [0]*len(eval(input_list))
for i, input in enumerate(eval(input_list)):
    if type(input) in [tuple, list]:
        output_list[i] = execute_function_from_string(code, *input)
    else:
        output_list[i] = execute_function_from_string(code, input)

In [36]:
print(output_list)

[[], [1, 0, 2, 0, 3, 0, 4], [10, -1, 20, -1, 30, -1, 40, -1, 50], [100, 999, 200, 999, 300], [-5, -100, -10, -100, -15, -100, -20]]


In [37]:
unittest = generate_unittest_advanced(code, code_explanation, input_list, output_list, openai_key)
print(unittest)

import unittest
from typing import List

def intersperse(numbers: List[int], delimeter: int) -> List[int]:
    if not numbers:
        return []

    result = []

    for n in numbers[:-1]:
        result.append(n)
        result.append(delimeter)

    result.append(numbers[-1])

    return result

class TestIntersperse(unittest.TestCase):

    def test_empty_list(self):
        self.assertEqual(intersperse([], 5), [])

    def test_positive_numbers(self):
        self.assertEqual(intersperse([1, 2, 3, 4], 0), [1, 0, 2, 0, 3, 0, 4])

    def test_positive_numbers_with_negative_delimiter(self):
        self.assertEqual(intersperse([10, 20, 30, 40, 50], -1), [10, -1, 20, -1, 30, -1, 40, -1, 50])

    def test_large_numbers_with_large_delimiter(self):
        self.assertEqual(intersperse([100, 200, 300], 999), [100, 999, 200, 999, 300])

    def test_negative_numbers_with_negative_delimiter(self):
        self.assertEqual(intersperse([-5, -10, -15, -20], -100), [-5, -100, -10, -100, -15, 

In [26]:
file_name = "my_code.py"
with open(file_name, "w") as file:
    file.write(unittest)

print(f"Python code has been written to '{file_name}'")

Python code has been written to 'my_code.py'
